# Creates the overall compound level heatplots, grouped by compound class and extraction/emission

### Load the libraries and data

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.cm as cm
import matplotlib.patches as patches
import matplotlib.pyplot as plt

In [2]:
# Load the data
DATA_EXCEL_PATH = ""
DATA_SHEET_NAME = ""
df = pd.read_excel(DATA_EXCEL_PATH, sheet_name=DATA_SHEET_NAME)

### Explore and process the data

In [3]:
# Do dilution aggregation
new_col = np.where(df['627811 LIQ 50x'] == 0, df['627811 LIQ 5x_Result'], df['627811 LIQ 50x'])
df.drop(columns='627811 LIQ 5x_Result')
df['627811 LIQ 50x'] = new_col

In [4]:
# Replace 0s with NaNs to color them separately
df.replace(0, np.nan, inplace=True)

In [ ]:
# How many NaN-only?
np.isnan(df.iloc[:, 10:-3]).all(axis=1).sum()

In [ ]:
# Drop NaN only
df.dropna(axis=1, how='all', inplace=True)
df = df[~np.isnan(df.iloc[:, 10:-3]).all(axis=1)]
orig_df = df.copy()
df.shape

In [ ]:
# See which master classes we have
df['Master Class'].unique()

In [8]:
# Rename classes for viewing simplicity
def convert_class(master_class):
    if master_class in {'aromatic - halogen', 'aromatic - halogen, nitrogen',
       'aromatic - halogen, nitrogen, oxygen',
       'aromatic - halogen, sulfur, nitrogen, oxygen',
       'aromatic - metallic', 'aromatic - nitrogen',
       'aromatic - nitrogen, oxygen', 'aromatic - oxygen',
       'aromatic - oxygen: phthalate', 'aromatic - phosphate',
       'aromatic - sulfur', 'aromatic - sulfur, halogen, nitrogen',
       'aromatic - sulfur, nitrogen',
       'aromatic - sulfur, nitrogen, oxygen', 'aromatic - sulfur, oxygen'}:
        return 'Aromatic heteroatom'
    elif master_class == 'aromatic hydrocarbon':
        return 'Aromatic hydrocarbon'
    elif master_class == 'glycol':
        return 'Glycol'
    elif master_class in {'hydrocarbon', 'hydrocarbon: n-alkane', 'hydrocarbon: terpene'}:
        return 'Hydrocarbon'
    elif master_class in {'long chain alcohol, acid, amide, ester or ether'}:
        return 'Long chain heteroatom'
    elif master_class in {'non-aromatic - halogen', 'non-aromatic - halogen, oxygen',
       'non-aromatic - halogen, phosphate',
       'non-aromatic - halogen, phosphorus', 'non-aromatic - nitrogen',
       'non-aromatic - nitrogen, oxygen', 'non-aromatic - oxygen',
       'non-aromatic - phosphate', 'non-aromatic - sulfur',
       'non-aromatic - sulfur, halogen, nitrogen, oxygen',
       'non-aromatic - sulfur, nitrogen',
       'non-aromatic - sulfur, nitrogen, oxygen'}:
        return 'Non-aromatic heteroatom'
    elif master_class == 'sterol and hopane':
        return 'Sterol/hopane'
    else:
        return 'Unknown/unclassified'

In [9]:
# Simplify for display purposes
df = df[~df['Master Class'].isin({'sulfur', 'siloxane', 'Vitamin E derivative', 'sterol and hopane'})]
df['Display Class'] = df['Master Class'].map(convert_class)
orig_df['Display Class'] = df['Master Class'].map(convert_class)
df.sort_values(by=['Display Class', 'RT1'], ascending=True, inplace=True)

In [ ]:
# Percentage of non-zero (non-NaN) data
100*np.count_nonzero(~np.isnan(df.iloc[:, 10:-3].values.ravel()))/df.iloc[:, 10:-3].values.size

In [ ]:
# See where we're at now
df

In [ ]:
# Get the absolute number of non-zero (non-NaN) compounds
np.count_nonzero((~np.isnan(orig_df.iloc[:, 10:-3]))), np.count_nonzero((~np.isnan(df.iloc[:, 10:-3])))

In [13]:
# Remove compounds that aren't present enough or that are in one of the ignored classes
filtered_df = orig_df[((~np.isnan(orig_df.iloc[:, 10:-3])).sum(axis=1) > 4) & ~orig_df['Master Class'].isin({'sulfur', 'siloxane', 'Vitamin E derivative', 'sterol and hopane'})]

In [ ]:
# Get how many are left
orig_df.shape, filtered_df.shape

In [16]:
# Get the indices of the different sections
cols = tuple(df.iloc[:, 10:-3].columns)
a1_39_liq = (cols.index('627343 LIQ 10x_Result'), cols.index('628248 LIQ 25x_Result')+1)
a1_39_ht = (cols.index('627343 HT 1x_A2'), cols.index('628248 HT 5x_A3')+1)
a1_39_lt = (cols.index('627343 LT 1x_Result'), cols.index('628248 LT 1x_Result')+1)
c1_21_ht = (cols.index('607144 HT 1x_A1'), cols.index('607877 D HT 1x_A2')+1)
c1_21_lt = (cols.index('607144 LT 1x_Result'), cols.index('607877 D LT 1x_Result')+1)
b1_21_ht = (cols.index('660783 HT 5x_A3'), cols.index('661142 HT 5x_A3')+1)
b1_21_lt = (cols.index('660783 LT 1x_Result'), cols.index('661142 LT 1x_Result')+1)

In [17]:
# Get the columns that correspond
a1_39_liq_cols = cols[slice(*a1_39_liq)]
a1_39_ht_cols = cols[slice(*a1_39_ht)]
a1_39_lt_cols = cols[slice(*a1_39_lt)]
b1_21_ht_cols = cols[slice(*b1_21_ht)]
b1_21_lt_cols = cols[slice(*b1_21_lt)]
c1_21_ht_cols = cols[slice(*c1_21_ht)]
c1_21_lt_cols = cols[slice(*c1_21_lt)]

In [ ]:
# Get the number non-zero (non-NaN)
np.count_nonzero((~np.isnan(filtered_df[list(a1_39_liq_cols)].iloc[:, 10:-3]))),\
np.count_nonzero((~np.isnan(filtered_df[list(a1_39_ht_cols + b1_21_ht_cols + c1_21_ht_cols)].iloc[:, 10:-3]))),\
np.count_nonzero((~np.isnan(filtered_df[list(a1_39_lt_cols + b1_21_lt_cols + c1_21_lt_cols)].iloc[:, 10:-3])))

In [ ]:
# How many per class?
filtered_df['Display Class'].value_counts()

### Rearrange the columns

In [20]:
# Sort the columns
df = df[list(tuple(df.columns[:10]) + a1_39_liq_cols + a1_39_ht_cols + a1_39_lt_cols + b1_21_ht_cols + b1_21_lt_cols + c1_21_ht_cols + c1_21_lt_cols + tuple(df.columns[-3:]))]

In [21]:
# Do it again so we have the new indices
cols = tuple(df.iloc[:, 10:-3].columns)
a1_39_liq = (cols.index('627343 LIQ 10x_Result'), cols.index('628248 LIQ 25x_Result')+1)
a1_39_ht = (cols.index('627343 HT 1x_A2'), cols.index('628248 HT 5x_A3')+1)
a1_39_lt = (cols.index('627343 LT 1x_Result'), cols.index('628248 LT 1x_Result')+1)
c1_21_ht = (cols.index('607144 HT 1x_A1'), cols.index('607877 D HT 1x_A2')+1)
c1_21_lt = (cols.index('607144 LT 1x_Result'), cols.index('607877 D LT 1x_Result')+1)
b1_21_ht = (cols.index('660783 HT 5x_A3'), cols.index('661142 HT 5x_A3')+1)
b1_21_lt = (cols.index('660783 LT 1x_Result'), cols.index('661142 LT 1x_Result')+1)

In [22]:
# Get the hex color for a given colormap and value
def get_hex(cmap, val):
    rgba = cmap(val)
    rgb = (int(rgba[0]*255), int(rgba[1]*255), int(rgba[2]*255))
    return '#%0.2X'%rgb[0] + '%0.2X'%rgb[1] + '%0.2X'%rgb[2]

### Make the colorbars and heatplots

In [29]:
as_np = df.iloc[:, 10:-3].values

In [45]:
cmap = cm.coolwarm

In [ ]:
# Make the colorbar
v = np.array([[np.log(np.nanquantile(as_np, 0.1)), np.log(np.nanquantile(as_np, 0.9))]])
plt.figure(figsize=(9, 1.5))
img = plt.imshow(v, cmap=cmap)
plt.gca().set_visible(False)
cax = plt.axes([0.1, 0.2, 0.8, 0.6])
cbar = plt.colorbar(orientation="horizontal", cax=cax, norm=matplotlib.colors.LogNorm(), ticks=[np.log(np.nanquantile(as_np, 0.1)), np.log(np.nanquantile(as_np, 0.5)), np.log(np.nanquantile(as_np, 0.9))])
cbar.ax.set_xticklabels([
    '{}\n(10ᵗʰ percentile)'.format(round(np.nanquantile(as_np, 0.1), 3)),
    '{}\n(50ᵗʰ percentile)'.format(round(np.nanquantile(as_np, 0.5), 3)),
    '{}\n(90ᵗʰ percentile)'.format(round(np.nanquantile(as_np, 0.9), 3))])
cbar.ax.set_title('Log-scaled estimated sample abundance in μg/g')
plt.text(0, 0.33, '   Max:\n254 μg/g')
plt.text(-6.5, 0.33, '      Min:\n5×10⁻⁵ μg/g')
# plt.savefig('colorbar.png', dpi=300, bbox_inches='tight')

In [ ]:
# A vertical version
v = np.array([[np.exp(-5), np.exp(-0.3)]])
plt.figure(figsize=(1.5, 9))
img = plt.imshow(v, cmap=cmap)
plt.gca().set_visible(False)
cax = plt.axes([0.1, 0.2, 0.8, 0.6])
plt.colorbar(orientation="vertical", cax=cax, label='Estimated concentration in μg/g', norm=matplotlib.colors.LogNorm())
# plt.savefig('colorbar.png', dpi=300, bbox_inches='tight')
np.exp(-5), np.exp(-0.3)

In [ ]:
# Make the full heatplot and label/color appropriately
a = np.log(df.iloc[:, 10:-3])
masked_array = np.ma.array(a, mask=np.isnan(a))
cmap.set_bad('black', 1.)
plt.imshow(masked_array, interpolation='nearest', aspect='auto', cmap=cmap, vmin=np.log(np.nanquantile(as_np, 0.1)), vmax=np.log(np.nanquantile(as_np, 0.9)))
for cls in ('Aromatic heteroatom', 'Aromatic hydrocarbon', 'Glycol', 'Hydrocarbon', 'Long chain heteroatom', 'Non-aromatic heteroatom'):
    plt.axhline(np.where(df['Display Class'] == cls)[0].max(), c='w')
for group in (a1_39_liq, a1_39_ht, a1_39_lt, b1_21_ht, b1_21_lt, c1_21_ht):
    plt.axvline(group[1], c='w')
cuf = patches.Rectangle((122, 380), 382, 110, zorder=-1, color='salmon', alpha=0.75)
rp = patches.Rectangle((505, 380), 112, 110, zorder=-1, color='skyblue', alpha=0.75)
plt.gcf().patches.extend([cuf, rp])
plt.text((a1_39_liq[0] + a1_39_liq[1])//2 - 8, -20, '  LIQ\nA1-39')
plt.text((a1_39_ht[0] + a1_39_ht[1])//2 - 8, -20, '   $\mathregular{T_H}$\nA1-39')
plt.text((a1_39_lt[0] + a1_39_lt[1])//2 - 8, -20, '   $\mathregular{T_L}$\nA1-39')
plt.text((b1_21_ht[0] + b1_21_ht[1])//2 - 8, -20, '   $\mathregular{T_H}$\nB1-21')
plt.text((b1_21_lt[0] + b1_21_lt[1])//2 - 8, -20, '   $\mathregular{T_H}$\nB1-21')
plt.text((c1_21_ht[0] + c1_21_ht[1])//2 - 8, -20, '   $\mathregular{T_H}$\nC1-21')
plt.text((c1_21_lt[0] + c1_21_lt[1])//2 - 8, -20, '   $\mathregular{T_H}$\nC1-21')
plt.text(36, -240, 'Clothing, upholstery, fabric (CUF)')
plt.text(178, -210, '   Rubber,\nplastic (RP)')
plt.text(-44, np.where(df['Display Class'] == 'Aromatic heteroatom')[0].max()-120, '  Aromatic\nheteroatom')
plt.text(-45, np.where(df['Display Class'] == 'Aromatic hydrocarbon')[0].max()-50, '  Aromatic\nhydrocarbon')
plt.text(-35, np.where(df['Display Class'] == 'Glycol')[0].max()-30, 'Glycol')
plt.text(-45, np.where(df['Display Class'] == 'Hydrocarbon')[0].max()-80, 'Hydrocarbon')
plt.text(-44, np.where(df['Display Class'] == 'Long chain heteroatom')[0].max()-50, ' Long chain\nheteroatom')
plt.text(-50, np.where(df['Display Class'] == 'Non-aromatic heteroatom')[0].max(), ' Non-aromatic\n  heteroatom')
plt.text(-44, np.where(df['Display Class'] == 'Unknown/unclassified')[0].max()-300, ' Unknown/\nunclassified')
plt.xticks([])
plt.yticks([])
#plt.savefig('Heatplot_1.png', bbox_inches='tight')

In [ ]:
# Make the first section of the heatplot and label/color appropriately
a = np.log(df.iloc[:920, 10:-3])
masked_array = np.ma.array(a, mask=np.isnan(a))
cmap.set_bad('black', 1.)
plt.imshow(masked_array, interpolation='nearest', aspect='auto', cmap=cmap, vmin=np.log(np.nanquantile(as_np, 0.1)), vmax=np.log(np.nanquantile(as_np, 0.9)))
for cls in ('Aromatic heteroatom', 'Aromatic hydrocarbon', 'Glycol'):
    plt.axhline(np.where(df['Display Class'] == cls)[0].max(), c='w')
for group in (a1_39_liq, a1_39_ht, a1_39_lt, b1_21_ht, b1_21_lt, c1_21_ht):
    plt.axvline(group[1], c='w')
cuf = patches.Rectangle((110, 380), 384, 110, zorder=-1, color='salmon', alpha=0.75)
rp = patches.Rectangle((495, 380), 112, 110, zorder=-1, color='skyblue', alpha=0.75)
plt.gcf().patches.extend([cuf, rp])
plt.text((a1_39_liq[0] + a1_39_liq[1])//2 - 8, -20, '  LIQ\nA1-39')
plt.text((a1_39_ht[0] + a1_39_ht[1])//2 - 8, -20, '   $\mathregular{T_H}$\nA1-39')
plt.text((a1_39_lt[0] + a1_39_lt[1])//2 - 8, -20, '   $\mathregular{T_L}$\nA1-39')
plt.text((b1_21_ht[0] + b1_21_ht[1])//2 - 8, -20, '   $\mathregular{T_H}$\nB1-21')
plt.text((b1_21_lt[0] + b1_21_lt[1])//2 - 8, -20, '   $\mathregular{T_H}$\nB1-21')
plt.text((c1_21_ht[0] + c1_21_ht[1])//2 - 8, -20, '   $\mathregular{T_H}$\nC1-21')
plt.text((c1_21_lt[0] + c1_21_lt[1])//2 - 8, -20, '   $\mathregular{T_H}$\nC1-21')
plt.text(36, -150, 'Clothing, upholstery, fabric (CUF)')
plt.text(178, -130, '   Rubber,\nplastic (RP)')
plt.text(-44, np.where(df['Display Class'] == 'Aromatic heteroatom')[0].max()-120, '  Aromatic\nheteroatom')
plt.text(-45, np.where(df['Display Class'] == 'Aromatic hydrocarbon')[0].max()-50, '  Aromatic\nhydrocarbon')
plt.text(-35, np.where(df['Display Class'] == 'Glycol')[0].max()-30, 'Glycol')
plt.text(-45, np.where(df['Display Class'] == 'Hydrocarbon')[0].max()-80, 'Hydrocarbon')
plt.xticks([])
plt.yticks([])
#plt.savefig('Heatplot_1.png', bbox_inches='tight')

In [ ]:
# Make the second section of the heatplot and label/color appropriately
a = np.log(df.iloc[920:, 10:-3])
masked_array = np.ma.array(a, mask=np.isnan(a))
cmap.set_bad('black', 1.)
plt.imshow(masked_array, interpolation='nearest', aspect='auto', cmap=cmap, vmin=np.log(np.nanquantile(as_np, 0.1)), vmax=np.log(np.nanquantile(as_np, 0.9)))
for cls in ('Long chain heteroatom', 'Non-aromatic heteroatom'):
    plt.axhline(np.where(df['Display Class'] == cls)[0].max()-920, c='w')
for group in (a1_39_liq, a1_39_ht, a1_39_lt, b1_21_ht, b1_21_lt, c1_21_ht):
    plt.axvline(group[1], c='w')
cuf = patches.Rectangle((122, 380), 383, 110, zorder=-1, color='salmon', alpha=0.75)
rp = patches.Rectangle((506, 380), 112, 110, zorder=-1, color='skyblue', alpha=0.75)
plt.gcf().patches.extend([cuf, rp])
plt.text((a1_39_liq[0] + a1_39_liq[1])//2 - 8, -20, '  LIQ\nA1-39')
plt.text((a1_39_ht[0] + a1_39_ht[1])//2 - 8, -20, '   $\mathregular{T_H}$\nA1-39')
plt.text((a1_39_lt[0] + a1_39_lt[1])//2 - 8, -20, '   $\mathregular{T_L}$\nA1-39')
plt.text((b1_21_ht[0] + b1_21_ht[1])//2 - 8, -20, '   $\mathregular{T_H}$\nB1-21')
plt.text((b1_21_lt[0] + b1_21_lt[1])//2 - 8, -20, '   $\mathregular{T_H}$\nB1-21')
plt.text((c1_21_ht[0] + c1_21_ht[1])//2 - 8, -20, '   $\mathregular{T_H}$\nC1-21')
plt.text((c1_21_lt[0] + c1_21_lt[1])//2 - 8, -20, '   $\mathregular{T_H}$\nC1-21')
plt.text(36, -150, 'Clothing, upholstery, fabric (CUF)')
plt.text(178, -130, '   Rubber,\nplastic (RP)')
plt.text(-44, np.where(df['Display Class'] == 'Long chain heteroatom')[0].max()-990, ' Long chain\nheteroatom')
plt.text(-50, np.where(df['Display Class'] == 'Non-aromatic heteroatom')[0].max()-950, ' Non-aromatic\n  heteroatom')
plt.text(-44, np.where(df['Display Class'] == 'Unknown/unclassified')[0].max()-1210, ' Unknown/\nunclassified')
plt.xticks([])
plt.yticks([])
# plt.savefig('Heatplot_2.png', bbox_inches='tight')